# LATAM in Numbers - Dashboard Project


### An overview of Latin America region, with some macro economic indicator

### GOAL
This notebook aims to collect and prepare the dataset to be used in the **Latam in Numbers Dashboard** project, which is supposed to visually showcase some key indicators of the Latin America region. 

Here are the key indicators to be initially collected for each country:
- Total Population
- Total GDP
- GDP per Capita - PPP
- Life Expectancy
- Happiness Index
- HDI

In [3]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print("✅ Imports completed!")

✅ Imports completed!


### Load the Dataset

In [4]:
file_path = "/home/rafael/code/latam_numbers/Latam_Numbers.xlsx"

df_xsl = pd.read_excel(file_path)
df_xsl

,Country ID,Country Name,Capital City
0,1,Argentina,Buenos Aires
1,2,Bolivia,Sucre
2,3,Brazil,Brasília
3,4,Chile,Santiago
4,5,Colombia,Bogotá
5,6,Costa Rica,San José
6,7,Cuba,Havana
7,8,Dominican Republic,Santo Domingo
8,9,Ecuador,Quito
9,10,El Salvador,San Salvador


In [5]:
# Adding HAITI to the table of countries:
df_start = pd.concat([df_xsl, pd.DataFrame({"Country Name": ["Haiti"],
                                            "Capital City": ["Port-Au-Prince"]})], ignore_index=True)

df_start.tail()

,Country ID,Country Name,Capital City
15,16.0,Paraguay,Asunción
16,17.0,Peru,Lima
17,18.0,Uruguay,Montevideo
18,19.0,Venezuela,Caracas
19,NaN,Haiti,Port-Au-Prince


In [6]:
# CLEANING THE TABLE

# Remove spaces at the beginning and at the end of country name:
df_start["Country Name"] = df_start["Country Name"].str.strip()

# Remove double or repeated spaces inside the text
df_start["Country Name"] = df_start["Country Name"].str.replace(r"\s+", " ", regex=True)

# Sorting names by Country (to acommodate Haiti correctly):
df_start = df_start.sort_values("Country Name")

df_start = df_start[["Country Name", "Capital City"]]

df_start

,Country Name,Capital City
0,Argentina,Buenos Aires
1,Bolivia,Sucre
2,Brazil,Brasília
3,Chile,Santiago
4,Colombia,Bogotá
5,Costa Rica,San José
6,Cuba,Havana
7,Dominican Republic,Santo Domingo
8,Ecuador,Quito
9,El Salvador,San Salvador


---

### New Column 'Country ISO Code':

In [7]:
# Let's create new column 'Country ISO Code':

iso_codes = {

 "Argentina": "AR",
 "Bolivia": "BO",
 "Brazil": "BR",
 "Chile": "CL",
 "Colombia": "CO",
 "Costa Rica": "CR",
 "Cuba": "CU",
 "Dominican Republic": "DO",
 "Ecuador": "EC",
 "El Salvador": "SV",
 "Guatemala": "GT",
 "Honduras": "HN",
 "Mexico": "MX",
 "Nicaragua": "NI",
 "Panama": "PA",
 "Paraguay": "PY",
 "Peru": "PE",
 "Uruguay": "UY",
 "Venezuela": "VE",
 "Haiti": "HT"
                }

In [8]:
# New column Country ISO Code (used for merging tables)
df_start["Country ISO Code"] = df_start["Country Name"].map(iso_codes)

df_start

,Country Name,Capital City,Country ISO Code
0,Argentina,Buenos Aires,AR
1,Bolivia,Sucre,BO
2,Brazil,Brasília,BR
3,Chile,Santiago,CL
4,Colombia,Bogotá,CO
5,Costa Rica,San José,CR
6,Cuba,Havana,CU
7,Dominican Republic,Santo Domingo,DO
8,Ecuador,Quito,EC
9,El Salvador,San Salvador,SV


In [9]:
# Build the country list string suitable for the WorldBank API
iso_list = df_start["Country ISO Code"].dropna().unique().tolist()
iso_csv = ";".join(code.lower() for code in iso_list)

iso_csv

'ar;bo;br;cl;co;cr;cu;do;ec;sv;gt;ht;hn;mx;ni;pa;py;pe;uy;ve'

---

## Total Population

In order to add a new column with **Total Population** by country, that's the workflow we'll follow:
1. Use the World Bank connector
2. Map the Country Name → ISO-2 country codes (World Bank prefers those)
3. Request the population indicator: 'SP.POP.TOTL'
4. Pull the most recent available value per country.
5. Finally merge that value back into df_start. 

In [10]:
# install required packages:
!pip install wbdata pycountry pandas

In [11]:
import wbdata # World Bank Data
from datetime import datetime
import requests

In [12]:
# Build the URL and call the World Bank API for the population indicator

indicator = "SP.POP.TOTL" # Id for Population
final_year = datetime.now().year - 1 # ie. 2024
start_year = final_year - 9    # last 10 years (2015–2024)

url = ( f"https://api.worldbank.org/v2/country/{iso_csv}/indicator/{indicator}"
        f"?date={start_year}:{final_year}&format=json&per_page=20000")

response = requests.get(url)
response.raise_for_status()   # stops if HTTP error
data = response.json()

rows = []

for rec in data[1]:    # payload[1] contains the actual observations
    iso2 = (rec.get("country") or {}).get("id")          # e.g. 'AR'
    country_name = (rec.get("country") or {}).get("value")
    year = int(rec.get("date")) if rec.get("date") else None
    pop = rec.get("value")   # can be None
    rows.append({
        "Country ISO Code": iso2.upper() if iso2 else None,
        "Country Name_API": country_name,
        "Year": year,
        "Total Population": pop
    })

df_wb_pop = pd.DataFrame(rows)

df_wb_pop.tail(20)

,Country ISO Code,Country Name_API,Year,Total Population
180,UY,Uruguay,2024,3386588
181,UY,Uruguay,2023,3388081
182,UY,Uruguay,2022,3390913
183,UY,Uruguay,2021,3396695
184,UY,Uruguay,2020,3398968
185,UY,Uruguay,2019,3397206
186,UY,Uruguay,2018,3394534
187,UY,Uruguay,2017,3388438
188,UY,Uruguay,2016,3379283
189,UY,Uruguay,2015,3368017


In [13]:
df_wb_pop.shape

(200, 4)

In [14]:
df_start

,Country Name,Capital City,Country ISO Code
0,Argentina,Buenos Aires,AR
1,Bolivia,Sucre,BO
2,Brazil,Brasília,BR
3,Chile,Santiago,CL
4,Colombia,Bogotá,CO
5,Costa Rica,San José,CR
6,Cuba,Havana,CU
7,Dominican Republic,Santo Domingo,DO
8,Ecuador,Quito,EC
9,El Salvador,San Salvador,SV


In [15]:
# Merge the 2 DataFrames:
df_merged = df_start.merge(df_wb_pop[["Country ISO Code", "Year", "Total Population"]],
                            on="Country ISO Code",
                            how="left"  )

# Reordering columns:
df_pop = df_merged[["Country Name","Country ISO Code","Capital City","Year","Total Population"]]

df_pop.tail(15)

,Country Name,Country ISO Code,Capital City,Year,Total Population
185,Uruguay,UY,Montevideo,2019,3397206
186,Uruguay,UY,Montevideo,2018,3394534
187,Uruguay,UY,Montevideo,2017,3388438
188,Uruguay,UY,Montevideo,2016,3379283
189,Uruguay,UY,Montevideo,2015,3368017
190,Venezuela,VE,Caracas,2024,28405543
191,Venezuela,VE,Caracas,2023,28300854
192,Venezuela,VE,Caracas,2022,28213017
193,Venezuela,VE,Caracas,2021,28237826
194,Venezuela,VE,Caracas,2020,28444077


✅ That's the final table with TOTAL POPULATION by country in the last 10 years.

---



## Total GDP

- Same flow as before, but now using a different Indicator ID for GDP: *NY.GDP.MKTP.CD*

In [16]:
indicator = "NY.GDP.MKTP.CD" # GDP (current US$)

url = ( f"https://api.worldbank.org/v2/country/{iso_csv}/indicator/{indicator}"
        f"?date={start_year}:{final_year}&format=json&per_page=20000")

response = requests.get(url)
response.raise_for_status()   # stops if HTTP error
data = response.json()

In [17]:
rows = []

for rec in data[1]:    # payload[1] contains the actual observations

    iso2 = (rec.get("country") or {}).get("id")          # e.g. 'AR'

    country_name = (rec.get("country") or {}).get("value")
    year = int(rec.get("date")) if rec.get("date") else None
    gdp_value = rec.get("value")   # GDP can be None if missing

    rows.append({
        "Country ISO Code": iso2.upper() if iso2 else None,
        #"Country Name": country_name,
        "Year": year,
        "Total GDP (USD)": gdp_value
                })


df_wb_gdp = pd.DataFrame(rows)

df_wb_gdp.head(20)

,Country ISO Code,Year,Total GDP (USD)
0,AR,2024,6.332667e+11
1,AR,2023,6.460753e+11
2,AR,2022,6.327901e+11
3,AR,2021,4.865641e+11
4,AR,2020,3.857405e+11
5,AR,2019,4.477547e+11
6,AR,2018,5.248199e+11
7,AR,2017,6.436284e+11
8,AR,2016,5.575323e+11
9,AR,2015,5.947493e+11


In [18]:
# Merging with df_pop
df_merged = df_pop.merge(df_wb_gdp[["Country ISO Code", "Year", "Total GDP (USD)"]],
                         on=["Country ISO Code", "Year"],
                         how="left")
df_gdp = df_merged
df_gdp.head(30)

,Country Name,Country ISO Code,Capital City,Year,Total Population,Total GDP (USD)
0,Argentina,AR,Buenos Aires,2024,45696159,6.332667e+11
1,Argentina,AR,Buenos Aires,2023,45538401,6.460753e+11
2,Argentina,AR,Buenos Aires,2022,45407904,6.327901e+11
3,Argentina,AR,Buenos Aires,2021,45312281,4.865641e+11
4,Argentina,AR,Buenos Aires,2020,45191965,3.857405e+11
5,Argentina,AR,Buenos Aires,2019,44973465,4.477547e+11
6,Argentina,AR,Buenos Aires,2018,44654882,5.248199e+11
7,Argentina,AR,Buenos Aires,2017,44288894,6.436284e+11
8,Argentina,AR,Buenos Aires,2016,43900313,5.575323e+11
9,Argentina,AR,Buenos Aires,2015,43477012,5.947493e+11


---

## GDP per Capita PPP

- Data also comes from **World Bank**
- Same flow as before, but now for another indicator: *NY.GDP.PCAP.PP.CD*

In [19]:
indicator = "NY.GDP.PCAP.PP.CD" # ID NY.GDP.MKTP.PP.CD

url = ( f"https://api.worldbank.org/v2/country/{iso_csv}/indicator/{indicator}"
        f"?date={start_year}:{final_year}&format=json&per_page=20000"        )

response = requests.get(url)
response.raise_for_status()
data = response.json()

In [20]:
rows = []

for rec in data[1]:    # payload[1] contains the actual observations

    iso2 = (rec.get("country") or {}).get("id")          # e.g. 'AR'

    country_name = (rec.get("country") or {}).get("value")
    year = int(rec.get("date")) if rec.get("date") else None
    gdp_capita = rec.get("value")   # GDP can be None if missing

    rows.append({
        "Country ISO Code": iso2.upper() if iso2 else None,
        "Country Name": country_name,
        "Year": year,
        "GDP per Capita, PPP (USD)": gdp_capita
                })

df_wb_capita = pd.DataFrame(rows)

df_wb_capita.head(30)

,Country ISO Code,Country Name,Year,"GDP per Capita, PPP (USD)"
0,AR,Argentina,2024,30175.535966
1,AR,Argentina,2023,30082.304525
2,AR,Argentina,2022,29597.693843
3,AR,Argentina,2021,26300.274261
4,AR,Argentina,2020,22393.347958
5,AR,Argentina,2019,23516.826198
6,AR,Argentina,2018,24410.391906
7,AR,Argentina,2017,23385.074090
8,AR,Argentina,2016,20105.761360
9,AR,Argentina,2015,19899.149928


In [21]:
# Merging with df_gdp:
df_merged2 = df_gdp.merge(df_wb_capita[["Country ISO Code", "Year", "GDP per Capita, PPP (USD)"]],
                          on=["Country ISO Code", "Year"],
                          how="left")

In [22]:
df_gdp_ppp = df_merged2

df_gdp_ppp.head()

,Country Name,Country ISO Code,Capital City,Year,Total Population,Total GDP (USD),"GDP per Capita, PPP (USD)"
0,Argentina,AR,Buenos Aires,2024,45696159,6.332667e+11,30175.535966
1,Argentina,AR,Buenos Aires,2023,45538401,6.460753e+11,30082.304525
2,Argentina,AR,Buenos Aires,2022,45407904,6.327901e+11,29597.693843
3,Argentina,AR,Buenos Aires,2021,45312281,4.865641e+11,26300.274261
4,Argentina,AR,Buenos Aires,2020,45191965,3.857405e+11,22393.347958


---


## HDI - Human Development Index

- Data was manually filtered and downloaded as Excel file from the Human Development Reports Data Center.
- Link: https://hdr.undp.org/data-center/documentation-and-downloads

In [23]:
df_hdi_raw = pd.read_excel('/home/rafael/code/latam_numbers/hdr-data.xlsx')

df_hdi_raw.head()

,countryIsoCode,country,indexCode,index,dimension,indicatorCode,indicator,year,value,note
0,ARG,Argentina,HDI,Human Development Index,NaN,hdi,Human Development Index (value),2015,0.859,NaN
1,ARG,Argentina,HDI,Human Development Index,NaN,hdi,Human Development Index (value),2016,0.857,NaN
2,ARG,Argentina,HDI,Human Development Index,NaN,hdi,Human Development Index (value),2017,0.861,NaN
3,ARG,Argentina,HDI,Human Development Index,NaN,hdi,Human Development Index (value),2018,0.861,NaN
4,ARG,Argentina,HDI,Human Development Index,NaN,hdi,Human Development Index (value),2019,0.861,NaN


In [24]:
df_hdi_raw["country"].value_counts()

country
Argentina                             9
Bolivia (Plurinational State of)      9
Uruguay                               9
El Salvador                           9
Paraguay                              9
Peru                                  9
Panama                                9
Nicaragua                             9
Mexico                                9
Haiti                                 9
Honduras                              9
Guatemala                             9
Ecuador                               9
Dominican Republic                    9
Cuba                                  9
Costa Rica                            9
Colombia                              9
Chile                                 9
Brazil                                9
Venezuela (Bolivarian Republic of)    9
Name: count, dtype: int64

In [ ]:
# We need to clean the value of BOLIVIA and VENEZUELA,
# so we can use this country column to merge
# with our other DataFrame:

df_hdi_raw["country"]  = df_hdi_raw["country"].replace({"Bolivia (Plurinational State of)": "Bolivia",
                                                        "Venezuela (Bolivarian Republic of)": "Venezuela"})

In [26]:
# CLEANING THE TABLE

# Remove spaces at the beginning and at the end of country name:
df_hdi_raw["country"] = df_hdi_raw["country"].str.strip()

# Remove double or repeated spaces inside the text
df_hdi_raw["country"] = df_hdi_raw["country"].str.replace(r"\s+", " ", regex=True)

print(df_hdi_raw["country"].unique())

['Argentina' 'Bolivia' 'Brazil' 'Chile' 'Colombia' 'Costa Rica' 'Cuba'
 'Dominican Republic' 'Ecuador' 'Guatemala' 'Honduras' 'Haiti' 'Mexico'
 'Nicaragua' 'Panama' 'Peru' 'Paraguay' 'El Salvador' 'Uruguay'
 'Venezuela']


In [27]:
# Lets rename the columns to be able to merge them:
df_hdi_raw = df_hdi_raw.rename(columns={"country":"Country Name",
                                        "year": "Year",
                                        "value": "HDI"})

df_hdi_raw[["Country Name", "Year", "HDI"]]

,Country Name,Year,HDI
0,Argentina,2015,0.859
1,Argentina,2016,0.857
2,Argentina,2017,0.861
3,Argentina,2018,0.861
4,Argentina,2019,0.861
...,...,...,...
175,Venezuela,2019,0.724
176,Venezuela,2020,0.699
177,Venezuela,2021,0.696
178,Venezuela,2022,0.706


In [28]:
# Merge with df_gdp_ppp
df_merged3 = df_gdp_ppp.merge(df_hdi_raw[["Country Name", "Year", "HDI"]],
                          on=["Country Name", "Year"],
                          how="left")

In [29]:
df_hdi = df_merged3

df_hdi.head(10)

,Country Name,Country ISO Code,Capital City,Year,Total Population,Total GDP (USD),"GDP per Capita, PPP (USD)",HDI
0,Argentina,AR,Buenos Aires,2024,45696159,6.332667e+11,30175.535966,NaN
1,Argentina,AR,Buenos Aires,2023,45538401,6.460753e+11,30082.304525,0.865
2,Argentina,AR,Buenos Aires,2022,45407904,6.327901e+11,29597.693843,0.858
3,Argentina,AR,Buenos Aires,2021,45312281,4.865641e+11,26300.274261,0.847
4,Argentina,AR,Buenos Aires,2020,45191965,3.857405e+11,22393.347958,0.851
5,Argentina,AR,Buenos Aires,2019,44973465,4.477547e+11,23516.826198,0.861
6,Argentina,AR,Buenos Aires,2018,44654882,5.248199e+11,24410.391906,0.861
7,Argentina,AR,Buenos Aires,2017,44288894,6.436284e+11,23385.074090,0.861
8,Argentina,AR,Buenos Aires,2016,43900313,5.575323e+11,20105.761360,0.857
9,Argentina,AR,Buenos Aires,2015,43477012,5.947493e+11,19899.149928,0.859


---

## Happiness Report

### >> From 2008 to 2023

- Data  taken from a **Kaggle dataset: World Happiness Report- 2024**
- Link: https://www.kaggle.com/datasets/jainaru/world-happiness-report-2024-yearly-updated 

In [30]:
df_happy_raw = pd.read_csv("/home/rafael/code/latam_numbers/World-happiness-report-updated_2024.csv", encoding="latin-1")

df_happy_raw.head()

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
0,Afghanistan,2008,3.724,7.350,0.451,50.5,0.718,0.164,0.882,0.414,0.258
1,Afghanistan,2009,4.402,7.509,0.552,50.8,0.679,0.187,0.850,0.481,0.237
2,Afghanistan,2010,4.758,7.614,0.539,51.1,0.600,0.118,0.707,0.517,0.275
3,Afghanistan,2011,3.832,7.581,0.521,51.4,0.496,0.160,0.731,0.480,0.267
4,Afghanistan,2012,3.783,7.661,0.521,51.7,0.531,0.234,0.776,0.614,0.268


In [31]:
 df_happy_raw = df_happy_raw.rename(columns={"Country name": "Country Name",
                                             "year": "Year",
                                             "Life Ladder": "Happiness Index"})

 df_happy_raw[["Country Name", "Year", "Happiness Index"]]

,Country Name,Year,Happiness Index
0,Afghanistan,2008,3.724
1,Afghanistan,2009,4.402
2,Afghanistan,2010,4.758
3,Afghanistan,2011,3.832
4,Afghanistan,2012,3.783
...,...,...,...
2358,Zimbabwe,2019,2.694
2359,Zimbabwe,2020,3.160
2360,Zimbabwe,2021,3.155
2361,Zimbabwe,2022,3.296


In [32]:
countries_list = df_start["Country Name"].unique()
print(countries_list)

['Argentina' 'Bolivia' 'Brazil' 'Chile' 'Colombia' 'Costa Rica' 'Cuba'
 'Dominican Republic' 'Ecuador' 'El Salvador' 'Guatemala' 'Haiti'
 'Honduras' 'Mexico' 'Nicaragua' 'Panama' 'Paraguay' 'Peru' 'Uruguay'
 'Venezuela']


In [36]:
# Filter df_happy_raw to only those countries in the list above:
df_happy_filtered = df_happy_raw[df_happy_raw["Country Name"].isin(countries_list)]

# Lets drop some columns and stick only to those we need:
df_happy_reduced = df_happy_filtered[["Country Name", "Year", "Happiness Index"]]
df_happy_reduced.head(5)

,Country Name,Year,Happiness Index
46,Argentina,2006,6.313
47,Argentina,2007,6.073
48,Argentina,2008,5.961
49,Argentina,2009,6.424
50,Argentina,2010,6.441


In [37]:
# Merge with df_hdi
df_merged4 = df_hdi.merge(df_happy_reduced[["Country Name", "Year", "Happiness Index"]],
                          on=["Country Name", "Year"],
                          how="left")

In [39]:
df_merged4.head(10)

,Country Name,Country ISO Code,Capital City,Year,Total Population,Total GDP (USD),"GDP per Capita, PPP (USD)",HDI,Happiness Index
0,Argentina,AR,Buenos Aires,2024,45696159,6.332667e+11,30175.535966,NaN,NaN
1,Argentina,AR,Buenos Aires,2023,45538401,6.460753e+11,30082.304525,0.865,6.393
2,Argentina,AR,Buenos Aires,2022,45407904,6.327901e+11,29597.693843,0.858,6.261
3,Argentina,AR,Buenos Aires,2021,45312281,4.865641e+11,26300.274261,0.847,5.908
4,Argentina,AR,Buenos Aires,2020,45191965,3.857405e+11,22393.347958,0.851,5.901
5,Argentina,AR,Buenos Aires,2019,44973465,4.477547e+11,23516.826198,0.861,6.086
6,Argentina,AR,Buenos Aires,2018,44654882,5.248199e+11,24410.391906,0.861,5.793
7,Argentina,AR,Buenos Aires,2017,44288894,6.436284e+11,23385.074090,0.861,6.039
8,Argentina,AR,Buenos Aires,2016,43900313,5.575323e+11,20105.761360,0.857,6.427
9,Argentina,AR,Buenos Aires,2015,43477012,5.947493e+11,19899.149928,0.859,6.697


### >> For 2024 only

- Data  taken from a **Kaggle dataset: World Happiness Report- 2024**
- Link: https://www.kaggle.com/datasets/jainaru/world-happiness-report-2024-yearly-updated 

In [50]:
path24 = "/home/rafael/code/latam_numbers/World-happiness-report-2024.csv"

df_happy24 = pd.read_csv(path24, encoding="latin-1")

df_happy24.head(5)

,Country name,Regional indicator,Ladder score,upperwhisker,lowerwhisker,Log GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Dystopia + residual
0,Finland,Western Europe,7.741,7.815,7.667,1.844,1.572,0.695,0.859,0.142,0.546,2.082
1,Denmark,Western Europe,7.583,7.665,7.500,1.908,1.520,0.699,0.823,0.204,0.548,1.881
2,Iceland,Western Europe,7.525,7.618,7.433,1.881,1.617,0.718,0.819,0.258,0.182,2.050
3,Sweden,Western Europe,7.344,7.422,7.267,1.878,1.501,0.724,0.838,0.221,0.524,1.658
4,Israel,Middle East and North Africa,7.341,7.405,7.277,1.803,1.513,0.740,0.641,0.153,0.193,2.298


In [51]:
df_happy24 = df_happy24.rename(columns={"Country name": "Country Name",
                                         "Ladder score": "Happiness Index"})

In [52]:
df_happy24.tail()

,Country Name,Regional indicator,Happiness Index,upperwhisker,lowerwhisker,Log GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Dystopia + residual
138,Congo (Kinshasa),Sub-Saharan Africa,3.295,3.462,3.128,0.534,0.665,0.262,0.473,0.189,0.072,1.102
139,Sierra Leone,Sub-Saharan Africa,3.245,3.366,3.124,0.654,0.566,0.253,0.469,0.181,0.053,1.068
140,Lesotho,Sub-Saharan Africa,3.186,3.469,2.904,0.771,0.851,0.000,0.523,0.082,0.085,0.875
141,Lebanon,Middle East and North Africa,2.707,2.797,2.616,1.377,0.577,0.556,0.173,0.068,0.029,-0.073
142,Afghanistan,South Asia,1.721,1.775,1.667,0.628,0.000,0.242,0.000,0.091,0.088,0.672


In [ ]:
# Adding new column for YEAR = 2024
df_happy24["Year"] = 2024

In [59]:
# Drop columns and use only the desired columns
df_happy24 = df_happy24[["Country Name", "Year", "Happiness Index"]]

In [60]:
print(countries_list)

['Argentina' 'Bolivia' 'Brazil' 'Chile' 'Colombia' 'Costa Rica' 'Cuba'
 'Dominican Republic' 'Ecuador' 'El Salvador' 'Guatemala' 'Haiti'
 'Honduras' 'Mexico' 'Nicaragua' 'Panama' 'Paraguay' 'Peru' 'Uruguay'
 'Venezuela']


In [64]:
# Filtering the dataframe for only LATAM countries in the list above
df_happy2_filtered = df_happy24[ df_happy24["Country Name"].isin(countries_list)   ]

df_happy2_filtered.sort_values("Country Name")

,Country Name,Year,Happiness Index
47,Argentina,2024,6.188
72,Bolivia,2024,5.784
43,Brazil,2024,6.272
37,Chile,2024,6.360
77,Colombia,2024,5.695
11,Costa Rica,2024,6.955
68,Dominican Republic,2024,5.823
73,Ecuador,2024,5.725
32,El Salvador,2024,6.469
41,Guatemala,2024,6.287


### 1. Trying MERGE:

In [ ]:
# Finally let's merge with the other df_merged4:
df_hap_test = df_merged4.merge(df_happy2_filtered,
                                on=["Country Name", "Year"],
                                how="left")

In [76]:
df_hap_test.head(3)

,Country Name,Country ISO Code,Capital City,Year,Total Population,Total GDP (USD),"GDP per Capita, PPP (USD)",HDI,Happiness Index_x,Happiness Index_y
0,Argentina,AR,Buenos Aires,2024,45696159,6.332667e+11,30175.535966,NaN,NaN,6.188
1,Argentina,AR,Buenos Aires,2023,45538401,6.460753e+11,30082.304525,0.865,6.393,NaN
2,Argentina,AR,Buenos Aires,2022,45407904,6.327901e+11,29597.693843,0.858,6.261,NaN


In [ ]:
# Cleaning the column Happiness Index after the merge
df_hap_test["Happiness Index"] = df_hap_test["Happiness Index_x"].fillna(df_hap_test["Happiness Index_y"])

# Drop the old _x and _y columns
df_happy = df_hap_test.drop(columns=["Happiness Index_x", "Happiness Index_y"])

df_happy.tail(30)

,Country Name,Country ISO Code,Capital City,Year,Total Population,Total GDP (USD),"GDP per Capita, PPP (USD)",HDI,Happiness Index
170,Peru,PE,Lima,2024,34217848,2.892220e+11,17802.418250,NaN,5.841
171,Peru,PE,Lima,2023,33845617,2.669587e+11,17011.227803,0.794,5.936
172,Peru,PE,Lima,2022,33475438,2.460656e+11,16669.156149,0.790,5.892
173,Peru,PE,Lima,2021,33155882,2.263288e+11,15280.551211,0.764,5.694
174,Peru,PE,Lima,2020,32838579,2.014097e+11,12563.303806,0.769,4.994
175,Peru,PE,Lima,2019,32449303,2.283460e+11,13563.537293,0.784,5.999
176,Peru,PE,Lima,2018,31897584,2.225970e+11,13078.232350,0.780,5.680
177,Peru,PE,Lima,2017,31324637,2.110080e+11,12591.856288,0.775,5.711
178,Peru,PE,Lima,2016,30866494,1.918981e+11,12066.546068,0.775,5.701
179,Peru,PE,Lima,2015,30457600,1.898030e+11,11600.387399,0.768,5.577


---

## Life Expectancy

- Data also comes from **World Bank**
- Same flow as before, but now for another indicator: *SP.DYN.LE00.IN*  → Life expectancy at birth, total (years)

In [81]:
indicator = "SP.DYN.LE00.IN"

url = ( f"https://api.worldbank.org/v2/country/{iso_csv}/indicator/{indicator}"
        f"?date={start_year}:{final_year}&format=json&per_page=20000"        )

response = requests.get(url)
response.raise_for_status()
data = response.json()

In [82]:
data

[{'page': 1,
  'pages': 1,
  'per_page': 20000,
  'total': 200,
  'sourceid': '2',
  'lastupdated': '2025-12-04'},
 [{'indicator': {'id': 'SP.DYN.LE00.IN',
    'value': 'Life expectancy at birth, total (years)'},
   'country': {'id': 'AR', 'value': 'Argentina'},
   'countryiso3code': 'ARG',
   'date': '2024',
   'value': None,
   'unit': '',
   'obs_status': '',
   'decimal': 0},
  {'indicator': {'id': 'SP.DYN.LE00.IN',
    'value': 'Life expectancy at birth, total (years)'},
   'country': {'id': 'AR', 'value': 'Argentina'},
   'countryiso3code': 'ARG',
   'date': '2023',
   'value': 77.395,
   'unit': '',
   'obs_status': '',
   'decimal': 0},
  {'indicator': {'id': 'SP.DYN.LE00.IN',
    'value': 'Life expectancy at birth, total (years)'},
   'country': {'id': 'AR', 'value': 'Argentina'},
   'countryiso3code': 'ARG',
   'date': '2022',
   'value': 75.806,
   'unit': '',
   'obs_status': '',
   'decimal': 0},
  {'indicator': {'id': 'SP.DYN.LE00.IN',
    'value': 'Life expectancy at bir

In [ ]:
rows = []

for rec in data[1]:    # payload[1] contains the actual observations

    iso2 = (rec.get("country") or {}).get("id")          # e.g. 'AR'

    country_name = (rec.get("country") or {}).get("value")
    year = int(rec.get("date")) if rec.get("date") else None
    life_exp = rec.get("value")   # GDP can be None if missing

    rows.append({
        "Country ISO Code": iso2.upper() if iso2 else None,
        "Country Name": country_name,
        "Year": year,
        "Life Expectancy (years)": life_exp
                })

df_wb_lifeexp = pd.DataFrame(rows)

df_wb_lifeexp.head(5)

,Country ISO Code,Country Name,Year,Life Expectancy (years)
0,AR,Argentina,2024,NaN
1,AR,Argentina,2023,77.395
2,AR,Argentina,2022,75.806
3,AR,Argentina,2021,73.948
4,AR,Argentina,2020,75.878


In [88]:
# Merge with df_happy
df_final = df_happy.merge(df_wb_lifeexp[["Country ISO Code", "Year", "Life Expectancy (years)"]],
                          on=["Country ISO Code", "Year"],
                          how="left")

In [89]:
df_final.head(50)

,Country Name,Country ISO Code,Capital City,Year,Total Population,Total GDP (USD),"GDP per Capita, PPP (USD)",HDI,Happiness Index,Life Expectancy (years)
0,Argentina,AR,Buenos Aires,2024,45696159,6.332667e+11,30175.535966,NaN,6.188,NaN
1,Argentina,AR,Buenos Aires,2023,45538401,6.460753e+11,30082.304525,0.865,6.393,77.395
2,Argentina,AR,Buenos Aires,2022,45407904,6.327901e+11,29597.693843,0.858,6.261,75.806
3,Argentina,AR,Buenos Aires,2021,45312281,4.865641e+11,26300.274261,0.847,5.908,73.948
4,Argentina,AR,Buenos Aires,2020,45191965,3.857405e+11,22393.347958,0.851,5.901,75.878
5,Argentina,AR,Buenos Aires,2019,44973465,4.477547e+11,23516.826198,0.861,6.086,76.847
6,Argentina,AR,Buenos Aires,2018,44654882,5.248199e+11,24410.391906,0.861,5.793,76.770
7,Argentina,AR,Buenos Aires,2017,44288894,6.436284e+11,23385.074090,0.861,6.039,76.543
8,Argentina,AR,Buenos Aires,2016,43900313,5.575323e+11,20105.761360,0.857,6.427,76.105
9,Argentina,AR,Buenos Aires,2015,43477012,5.947493e+11,19899.149928,0.859,6.697,76.600


✅ Final dataframe created successfully! Now just export it to a CSV file:

## Final Export

In [ ]:
df_final.to_csv("latam_numbers.csv", sep=";", index=False)

---